### This notebook is target to explore the mechanism of Fitness class in DEAP
[DEAP - Distributed Evolutionary Algorithms in Python](https://github.com/DEAP/deap)

In [1]:
from deap import tools, creator, base
import pdb
from operator import mul, truediv
from collections import Sequence

In [2]:
class Fitness(object):
    """The fitness is a measure of quality of a solution. If *values* are
    provided as a tuple, the fitness is initalized using those values,
    otherwise it is empty (or invalid).

    :param values: The initial values of the fitness as a tuple, optional.

    Fitnesses may be compared using the ``>``, ``<``, ``>=``, ``<=``, ``==``,
    ``!=``. The comparison of those operators is made lexicographically.
    Maximization and minimization are taken care off by a multiplication
    between the :attr:`weights` and the fitness :attr:`values`. The comparison
    can be made between fitnesses of different size, if the fitnesses are
    equal until the extra elements, the longer fitness will be superior to the
    shorter.

    Different types of fitnesses are created in the :ref:`creating-types`
    tutorial.

    .. note::
       When comparing fitness values that are **minimized**, ``a > b`` will
       return :data:`True` if *a* is **smaller** than *b*.
    """

    weights = None
    """The weights are used in the fitness comparison. They are shared among
    all fitnesses of the same type. When subclassing :class:`Fitness`, the
    weights must be defined as a tuple where each element is associated to an
    objective. A negative weight element corresponds to the minimization of
    the associated objective and positive weight to the maximization.

    .. note::
        If weights is not defined during subclassing, the following error will
        occur at instantiation of a subclass fitness object:

        ``TypeError: Can't instantiate abstract <class Fitness[...]> with
        abstract attribute weights.``
    """

    wvalues = ()
    """Contains the weighted values of the fitness, the multiplication with the
    weights is made when the values are set via the property :attr:`values`.
    Multiplication is made on setting of the values for efficiency.

    Generally it is unnecessary to manipulate wvalues as it is an internal
    attribute of the fitness used in the comparison operators.
    """

    def __init__(self, values=()):
        pdb.set_trace()
        if self.weights is None:
            raise TypeError("Can't instantiate abstract %r with abstract "
                            "attribute weights." % (self.__class__))

        if not isinstance(self.weights, Sequence):
            raise TypeError("Attribute weights of %r must be a sequence."
                            % self.__class__)

        if len(values) > 0:
            self.values = values

    def getValues(self):
        return tuple(map(truediv, self.wvalues, self.weights))

    def setValues(self, values):
        pdb.set_trace()
        try:
            self.wvalues = tuple(map(mul, values, self.weights))
        except TypeError:
            _, _, traceback = sys.exc_info()

    def delValues(self):
        self.wvalues = ()

    values = property(getValues, setValues, delValues,
                      ("Fitness values. Use directly ``individual.fitness.values = values`` "
                       "in order to set the fitness and ``del individual.fitness.values`` "
                       "in order to clear (invalidate) the fitness. The (unweighted) fitness "
                       "can be directly accessed via ``individual.fitness.values``."))

    def dominates(self, other, obj=slice(None)):
        """Return true if each objective of *self* is not strictly worse than
        the corresponding objective of *other* and at least one objective is
        strictly better.

        :param obj: Slice indicating on which objectives the domination is
                    tested. The default value is `slice(None)`, representing
                    every objectives.
        """
        not_equal = False
        for self_wvalue, other_wvalue in zip(self.wvalues[obj], other.wvalues[obj]):
            if self_wvalue > other_wvalue:
                not_equal = True
            elif self_wvalue < other_wvalue:
                return False
        return not_equal

    @property
    def valid(self):
        """Assess if a fitness is valid or not."""
        return len(self.wvalues) != 0

    def __hash__(self):
        return hash(self.wvalues)

    def __gt__(self, other):
        return not self.__le__(other)

    def __ge__(self, other):
        return not self.__lt__(other)

    def __le__(self, other):
        pdb.set_trace()
        return self.wvalues <= other.wvalues

    def __lt__(self, other):
        pdb.set_trace()
        return self.wvalues < other.wvalues

    def __eq__(self, other):
        return self.wvalues == other.wvalues

    def __ne__(self, other):
        return not self.__eq__(other)

    def __deepcopy__(self, memo):
        """Replace the basic deepcopy function with a faster one.

        It assumes that the elements in the :attr:`values` tuple are
        immutable and the fitness does not contain any other object
        than :attr:`values` and :attr:`weights`.
        """
        copy_ = self.__class__()
        copy_.wvalues = self.wvalues
        return copy_

    def __str__(self):
        """Return the values of the Fitness object."""
        return str(self.values if self.valid else tuple())

    def __repr__(self):
        """Return the Python code to build a copy of the object."""
        return "%s.%s(%r)" % (self.__module__, self.__class__.__name__,
                              self.values if self.valid else tuple())

## Single-objective fitness

In [3]:
creator.create("Fitness", Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

In [4]:
# trivial evluation function
def evaluate(individual):
    return 1,

# trivial initialization function
def constant(n):
    return n,

In [5]:
toolbox = base.Toolbox()
toolbox.register("attr", constant, 1)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr)
toolbox.register("evaluate", evaluate)

In [6]:
# initialization (__init__)
ind1 = toolbox.individual()
ind2 = toolbox.individual()

> <ipython-input-2-6b5fb5797020>(50)__init__()
-> if self.weights is None:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(50)__init__()
-> if self.weights is None:
(Pdb) c


In [7]:
# trivial assignment of fitness (setValues() of Fitness class will be called)
ind1.fitness.values = toolbox.evaluate(ind1)
ind2.fitness.values = toolbox.evaluate(ind2)

> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c


### Manually assign the fitnesses

In [8]:
# setValue() function of Fitness class will be called
ind1.fitness.values = (6,)
ind2.fitness.values = (3,)

> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c


### Fitness comparsion (comparing weighted fitness implicitly)

In [9]:
# it is actually comparing the weighted fitness
result = ind1.fitness < ind2.fitness
print('comparing fitness {} < {} : {}'.format(ind1.fitness, ind2.fitness, result))
print('Actually, it is comparing weighted fitness {} < {} : {}'.format(ind1.fitness.wvalues, ind2.fitness.wvalues, result))

> <ipython-input-2-6b5fb5797020>(117)__lt__()
-> return self.wvalues < other.wvalues
(Pdb) c
comparing fitness (6.0,) < (3.0,) : True
Actually, it is comparing weighted fitness (-6.0,) < (-3.0,) : True


### Max comparison (calling le function implicitly)

In [10]:
result = max(ind1.fitness, ind2.fitness)
print('return of max({} < {}) : {}'.format(ind1.fitness, ind2.fitness, result))
print('Actually, it equals to max({} < {}) : {}'.format(ind1.fitness.wvalues, ind2.fitness.wvalues, result))

> <ipython-input-2-6b5fb5797020>(113)__le__()
-> return self.wvalues <= other.wvalues
(Pdb) c
return of max((6.0,) < (3.0,)) : (3.0,)
Actually, it equals to max((-6.0,) < (-3.0,)) : (3.0,)


# Multi-objective fitness

In [11]:
creator.create("Fitness", Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.Fitness)

/Users/yao/anaconda3/lib/python3.6/site-packages/deap-1.1.0-py3.6-macosx-10.9-x86_64.egg/deap/creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/yao/anaconda3/lib/python3.6/site-packages/deap-1.1.0-py3.6-macosx-10.9-x86_64.egg/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [12]:
# trivial evluation function
def evaluate(individual):
    return 1, 1

# trivial initialization function
def constant(n):
    return n,

In [13]:
toolbox = base.Toolbox()
toolbox.register("attr", constant, 1)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr)
toolbox.register("evaluate", evaluate)

###  __init__() function of Fitness class will be processed by using registered toolbox

In [14]:
ind1 = toolbox.individual()
ind2 = toolbox.individual()

> <ipython-input-2-6b5fb5797020>(50)__init__()
-> if self.weights is None:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(50)__init__()
-> if self.weights is None:
(Pdb) c


###  setValues() function of Fitness class will be processed when the fitness is assigned as follows, also:
- Raw *fitness* will be multipled with its specified property of *weights* [e.g. (-1.0, 1)] here
- Then stored as *self.wvalues*  (weighted fitness)
- Thus you can optimize a *maximization* problem with *weight* of 1.0
- And a *minimization* problem with weight of -1.0

In [15]:
# trivial assignmnet of fitness
ind1.fitness.values = toolbox.evaluate(ind1)
ind2.fitness.values = toolbox.evaluate(ind2)

> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c


### Manually assigment of fitness to show domination

In [16]:
# ind1 dominates ind2 in terms of fitness
ind1.fitness.values = (3, 9)
ind2.fitness.values = (7, 8)

> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c


In [17]:
# note that we use weights (-1.0, 1.0)
# it means to minimize the first objective and maximize the second objective
print('{} dominates {}:'.format(ind1.fitness.values, ind2.fitness.values), ind1.fitness.dominates(ind2.fitness))
print('Cause: first objective: {}>{}:{}, and second objective: {}>{}:{}'.format(ind1.fitness.wvalues[0], ind2.fitness.wvalues[0],
                                                                                ind1.fitness.wvalues[0] > ind2.fitness.wvalues[0],
                                                                         ind1.fitness.wvalues[1], ind2.fitness.wvalues[1],
                                                                               ind1.fitness.wvalues[1] > ind2.fitness.wvalues[1]))
print('Thus, {} dominates {}:'.format(ind2.fitness.values, ind1.fitness.values), ind2.fitness.dominates(ind1.fitness))

(3.0, 9.0) dominates (7.0, 8.0): True
Cause: first objective: -3.0>-7.0:True, and second objective: 9.0>8.0:True
Thus, (7.0, 8.0) dominates (3.0, 9.0): False


In [18]:
# ind1 and ind2 are non-dominated to each other in terms of fitness
ind1.fitness.values = (8, 9)
ind2.fitness.values = (7, 8)

# note that we use weights (-1.0, 1.0)
# it means to minimize the first objective and maximize the second objective
print('{} dominates {}:'.format(ind1.fitness.values, ind2.fitness.values), ind1.fitness.dominates(ind2.fitness))

print('Cause: first objective: {}>{}:{}, and second objective: {}>{}:{}'.format(ind1.fitness.wvalues[0], ind2.fitness.wvalues[0],
                                                                                ind1.fitness.wvalues[0] > ind2.fitness.wvalues[0],
                                                                         ind1.fitness.wvalues[1], ind2.fitness.wvalues[1],
                                                                               ind1.fitness.wvalues[1] > ind2.fitness.wvalues[1]))

print('Thus, {} dominates {}:'.format(ind2.fitness.values, ind1.fitness.values), ind2.fitness.dominates(ind1.fitness))

> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
> <ipython-input-2-6b5fb5797020>(66)setValues()
-> try:
(Pdb) c
(8.0, 9.0) dominates (7.0, 8.0): False
Cause: first objective: -8.0>-7.0:False, and second objective: 9.0>8.0:True
Thus, (7.0, 8.0) dominates (8.0, 9.0): False
